In [1]:
import numpy as np

import MPO_hyperparameter_v2
import MPO_tt_v2
import MPO_inference_v2
import MPO_train_v2 as MPO

In [2]:
# List of layer 1 weights (0-3), layer 1 bias (4), layer 2 weights (5-8), layer 2 bias (9)
weights = MPO.main()

Instructions for updating:
Please use alternatives such as: tensorflow_datasets.load('mnist')
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/_DataSet.py from tensorflow/models.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Accuracy at step 0: 0.1394
Accuracy at step 1: 0.1733
Accuracy at step 2: 0.1769
Accuracy at step 3: 0.1828
Accuracy at step 4: 0.2035
Accuracy at step 5: 0.2273
Accuracy at step 8: 0.2566
Accuracy at step 9: 0.2708
After 9 trainning step(s),best acc

In [3]:
def weights_to_MPO(weights, D, sigma, sigma_prime, layer):
    # Reshape weights, order of indices are an assumption and may need to be modified
    MPO = []
    num_sites = len(sigma)
    starting_site = layer*(num_sites+1)  # +1 to skip bias

    for i in range(num_sites):
        if i == 0:
            # Weights[0] (32, 4) = (D*sigma', sigma) or (sigma'*D, sigma)
            site = np.reshape(weights[i+starting_site], (D, sigma_prime[i], sigma[i]))
            # Need shape (D_right, sigma, sigma')
            site = np.transpose(site, (0, 2, 1))
        elif i != 0 and i != num_sites-1:
            # Weights[1] (32, 56) = (D*sigma', D*sigma) or (sigma'*D, sigma*D)
            # First D is right bond, Second D is left bond
            site = np.reshape(weights[i+starting_site], (D, sigma_prime[i], D, sigma[i]))
            # Need shape (D_left, D_right, sigma, sigma')
            site = np.transpose(site, (2, 0, 3, 1))
        elif i == num_sites-1:
            # Weights[2] (4, 32) = (sigma', D*sigma) or (sigma', sigma*D)
            # First D is right bond, Second D is left bond
            site = np.reshape(weights[i+starting_site], (sigma_prime[i], D, sigma[i]))
            # Need shape (D_left, sigma, sigma')
            site = np.transpose(site, (1, 2, 0))
        MPO.append(site)
    bias = weights[num_sites+starting_site]
    return MPO, bias

In [4]:
D = 2
sigma = [4, 7, 7, 4]
sigma_prime = [4, 4, 4, 4]
MPO_0, bias_0 = weights_to_MPO(weights, D, sigma, sigma_prime, layer=0)

In [5]:
D = 2
sigma = [4, 4, 4, 4]
sigma_prime = [1, 10, 1, 1]
MPO_1, bias_1 = weights_to_MPO(weights, D, sigma, sigma_prime, layer=1)

In [6]:
import contractions as con
temp = con.contract_horizontal(MPO_0[0], MPO_0[1], 'right')
temp = con.contract_horizontal(temp, MPO_0[2], 'right')
layer_0 = con.contract_horizontal(temp, MPO_0[3], 'right')

temp = con.contract_horizontal(MPO_1[0], MPO_1[1], 'right')
temp = con.contract_horizontal(temp, MPO_1[2], 'right')
layer_1 = con.contract_horizontal(temp, MPO_1[3], 'right')

In [7]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

data = input_data.read_data_sets("./data/",validation_size=0,one_hot=True)

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz


In [12]:
(data.test.images[test_point] @ layer_0).shape

(256,)

In [8]:
# Same as random guessing
import activation_functions as act
total = 10000
correct = 0
for test_point in range(0, total):
    xW_1 = data.test.images[test_point] @ layer_0 + bias_0
    xW_1 = act.ReLU(xW_1)
    result = xW_1 @ layer_1 + bias_1

    index = (np.abs(result-0)).argmin()
    if index == list(data.test.labels[test_point]).index(1):
        correct += 1

print(correct/total*100, "%")

6.1899999999999995 %


In [6]:
def apply_layer(x, W, bias, activation='linear'):
    xW = []
    for i in range(len(x)):
        if i == 0 or i == len(W)-1:
            # Contract from bottom
            site = np.einsum('ij, abi->ajb', x[i], W[i])
            # Transpose since we move from MPO to MPS index notation
            site = np.reshape(site, (W[i].shape[0]*x[i].shape[1], W[i].shape[1]))
            site = site.T
            xW.append(site)
        else:
            # Contract from bottom
            site = np.einsum('ijk, abkd->iajbd', x[i], W[i])
            site = np.reshape(site, (x[i].shape[0]*W[i].shape[0], x[i].shape[1]*W[i].shape[1], W[i].shape[3]))
            xW.append(site)
    return xW

In [7]:
x1 = np.random.rand(1, 4)
x2 = np.random.rand(1, 1, 7)
x3 = np.random.rand(1, 1, 7)
x4 = np.random.rand(1, 4)

x = [x1, x2, x3, x4]

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz


9.99 %
